# クラスタリング

クラスタリングは教師無し学習の一種で、似たデータをひとまとめにすることが出来ます。

クラスタリングを行う場合、データをクラスタリングすることが本当に主目的の場合が多いと思います。が、他の解析手段の前処理として似たデータだけを纏めてデータを小さく分割してから目的となる解析を行う場合もあるでしょう。

## データの前処理

クラスタリングを使う場合は、

1. アルゴリズム選択
2. 類似度や距離の選択
3. データの前処理
4. クラスタ数

について考慮すべきです。

#### アルゴリズム選択

scikit-learn で用意されている[クラスタリングのアルゴリズムの一覧のページ](http://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods)があります。
アルゴリズムによって得手不得手があるので、それを理解して使い分けるのが重要です。

データ量が多い（10万以上）場合には、（計算速度的な問題で）使用可能なアルゴリズムは、KMeans法（MiniBatchKMeans法）ぐらいしか使い物にならないような気もします。

しかし、KMeans法はデータの属性値の「平均値」に意味がある場合にしか使いにくいです。例えば性別のようなカテゴリカル変数の場合、平均値付近にそもそもデータは存在しません。

また、ユーザの購買行動でユーザをクラスタリングするような場合、よくあるRFM(Recency-Frequency-Monetary, 最近の購買からの時間、購買頻度、購買金額)の値と、個々のアイテムに関する購買個数（あるいはそれのTf-Idf）の値と、単純に並べてベクトルにするのが適切か考えてみましょう。識別問題と異なり、クラスタリングの場合は「重み」を学習で定めないため、それらを混ぜてクラスタリングするのは難しいように思います。

#### 類似度や距離の選択

KMeans法であればユークリッド距離を前提としていますし、前述のように平均が意味がある必要があります。対して、多くの階層クラスタリングの場合は距離ではなく類似度を用いて計算できます。

距離も、普通にユークリッド距離($L^2$)を使うのが良い場合もあれば、マンハッタン距離($L^1$)、チェビシェフ距離($L^{\infty}$)、さらにそれらへの正規化の有無、あるいはJaccard距離やコサイン距離などを選ぶべき場合もあります。

適切と思われるものを選んでください。

#### データの前処理

高次元データのクラスタリングは、高次元データの識別問題よりも更に難しいように思われます。まず可視化して結果を確認しにくいことと、いわゆる「次元の呪い」により個々のデータが類似しないからです。

購入アイテムの類似度を用いてユーザをクラスタリングする場合など、購入アイテム履歴を単純に素性化すると高次元になってしまいますが、適切な方法で次元削減するか、Biclustringを使うなど工夫すべきでしょう。

# 各種アルゴリズムの使い方

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## Irisデータのクラスタリング

定番のIrisデータセットをクラスタリングしてみます。まずデータを読み込みます。
irisデータは４次元データですが、データの特徴は最初の２つの値で大体わかるので、プロットはその値で行うことにします。

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
color = ['gx', 'ro', 'b*']
for c in range(3):
    xs = [d[0] for d,t in zip(iris.data, iris.target) if t==c]
    ys = [d[1] for d,t in zip(iris.data, iris.target) if t==c]
    plt.plot(xs, ys, color[c])
plt.show()

# KMeans

教科書などに良く載っているKMeans法はデータ量が比較的多い場合でも使用できる汎用のクラスタリング手法です。とりあえずクラスタリングしてみたい、という場合にはこれを試すのが良いでしょう。

データ量が数万程度であれば [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) アルゴリズムを使用できますが、データ数もクラスタ数も多い場合は、[MiniBatchKMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html)の使用も検討するのが良いでしょう。

更にデータ量が多い（例えば10万件を超えるとか）場合は、MiniBatchKMeansアルゴリズムを使い、

* 例えばデータを偏りがないように10万件以下となるように分割する。
* 最初の分割データを使って `fit` メソッドで学習する。
* 残りの分割データのそれぞれについて、`partial_fit` メソッドでデータを追加学習させる。

という方法を取ることで、クラスタリング時間を短縮できます。（`partial_fit`は既存のクラスタにデータを追加してクラスタ中心の移動は行いますが、最初のクラスタ中心の選択は`fit`でしか行いません。また、`MiniBatchKMeans`はそもそも全データを使って最初のクラスタ中心を計算しません。詳しくは[MiniBatchKMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html)の解説参照。）

KMeans法の場合、クラスタ数を最初に決める必要があります。予めクラスタ数が判っていたり、後述のSilhouette scoreなどを用いて決めるなどの方法もありますが、なんのヒントもない場合に最初に試す値としては
$$\text{クラスタ数} \sim \sqrt{\frac{\text{アイテム数}}{2}}$$
を試してみると良いでしょう(リンク：[Wikipedia記事](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set#Rule_of_thumb))。
（Silhouette scoreの計算はアイテム数が増えると時間がかかるため、アイテム数が多い場合はあまり実用的ではないです。）

In [ ]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1)
km.fit(iris.data)

クラスタリングの結果を得るには `predict` メソッドを使用します。引数は必ずしも学習データである必要はありません。

In [ ]:
p = km.predict(iris.data)
for c in range(3):
    xs = [d[0] for d,t in zip(iris.data, p) if t==c]
    ys = [d[1] for d,t in zip(iris.data, p) if t==c]
    plt.plot(xs, ys, color[c])
plt.show()

クラスタリングの良し悪しは幾つかのメトリクスで評価できます。正解が解っている場合、V-measure, Rand-Index などの指標を使うことも可能です。

実際に使用する場合は、そもそも正解が分からない場合が多いでしょう。その場合に使用可能な指標に、Silhouette scoreがあります。これは、平均クラスタ内距離($=a$)と、平均クラスタ間距離($=b$)を用いて、$(b - a) / \textrm{max}(a, b)$ で求める値です。

In [ ]:
from sklearn import metrics
print("V-measure: %0.3f" % metrics.v_measure_score(iris.target, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(iris.target, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(iris.data, km.labels_, sample_size=1000))

クラスタ数を変えてSilhouette scoreを求め、最適クラスタ数を求めてみましょう。

In [ ]:
n_clusters = range(2, 10)
silhouettes = []
for n_cluster in n_clusters:
    km = KMeans(n_clusters=n_cluster, init='k-means++', max_iter=100, n_init=1)
    km.fit(iris.data)
    silhouettes.append(metrics.silhouette_score(iris.data, km.labels_))
plt.plot(np.array(n_clusters), np.array(silhouettes), 'o-')

グラフから見てわかるように、またそもそも散布図から見てわかるように、正解なしで推定したクラスタは２個の様です。あるいは５個でしょうか。下記のコードで n_cluster の値を変えて図を眺めてみてください。

In [ ]:
n_cluster = 5
km = KMeans(n_clusters=n_cluster, init='k-means++', max_iter=100, n_init=1)
km.fit(iris.data)
p = km.predict(iris.data)
for c in range(n_cluster):
    xs = [d[0] for d,t in zip(iris.data, p) if t==c]
    ys = [d[1] for d,t in zip(iris.data, p) if t==c]
    plt.plot(xs, ys, 'o')
plt.show()

## 色々なアルゴリズム

アルゴリズムを変更して結果を眺めやすいように、幾つか関数を準備します。

In [ ]:
def draw_silhouette(make_alg, n_min, n_max):
    """
    args:
        make_alg: クラスタリングアルゴリズムのインスタンスを作る関数。引数はクラスタ数。
        n_min, n_max: クラスタ数を変化させる範囲
    """
    n_clusters = range(n_min, n_max)
    silhouettes = []
    for n_cluster in n_clusters:
        alg = make_alg(n_cluster)
        alg.fit(iris.data)
        silhouettes.append(metrics.silhouette_score(iris.data, alg.labels_))
    plt.plot(np.array(n_clusters), np.array(silhouettes), 'o-')
    plt.show()

def plot_scatter(alg, n_cluster):
    """
    args:
        alg: クラスタリングアルゴリズム
        n_cluster: クラスタ数
    """
    alg.fit(iris.data)
    p = alg.predict(iris.data)
    for c in range(n_cluster):
        xs = [d[0] for d,t in zip(iris.data, p) if t==c]
        ys = [d[1] for d,t in zip(iris.data, p) if t==c]
        plt.plot(xs, ys, 'o')
    plt.show()

KMeans法であればこんな感じになります。

In [ ]:
def make_kmeans(n_cluster):
    return KMeans(n_clusters=n_cluster, init='k-means++', max_iter=100, n_init=1)

draw_silhouette(make_kmeans, 2, 10)
plot_scatter(make_kmeans(6), 6)

### 課題

[2.3.1. Overview of clustering methods](http://scikit-learn.org/stable/modules/clustering.html#overview-of-clustering-methods)を見て、KMeans以外のクラスタリングを試してみてください。

あるいは、[Comparing different clustering algorithms on toy datasets](http://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html#example-cluster-plot-cluster-comparison-py)のソースコードをコピペして動かしてみるとか試してみてください。また、このソースコードを見ると、上の課題の `make_xxxx` に相当するクラスタリングアルゴリズムのオブジェクトを生成する箇所の参考になると思います。

# 階層型クラスタリング

データの数がある程度以下の場合は、階層クラスタリングが使用できます。階層クラスタリングの場合は、デンドログラム(樹形樹)を書かせることができます。

デンドログラムを書かせる場合は、scikit-learnよりscipyの方が使いやすいかと思います。

In [ ]:
import scipy.spatial.distance as distance
from scipy.cluster.hierarchy import linkage, dendrogram

`scipy.cluster.hierarchy` を使う場合、アイテム間の距離を与える行列を用いてクラスタリングさせるのが基本となりますが、アイテムの配列を与えて計算させることも可能です。

`linkage` に与える引数のうち、`method` にはクラスタ間の距離の定義を指定します。詳しくは[scipy.cluster.hierarchy.linkage](http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.linkage.html)を読んでください。無難にはWard法を指定すれば良いのではないかと思いますが、連結効果の要/不要や、クラスタサイズを大きくしたい/小さくしたいなど、使用目的によって変える必要があるかと思います。

`metric` は距離の計算方法です。今回の様に多次元正規分布的な場合はユークリッド距離で良いと思いますが、テキスト処理や購買履歴など（＝ベクトルの大きさに影響されたくない）場合はコサイン距離を使う、0/1的なカテゴリカル変数が多いなどの場合はジャカード距離を使う、など工夫する必要があります。使えるものは[scipy.spatial.distance.pdist](http://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html)に書かれています。

今回は `dendrogram` の `labels` 引数に正解(int値)を渡します。戻り値(dict型)の`ivl`属性に、`labels`を並べ替えた値が入っています。（重なってよく読めませんが、画面下の方に表示されています。）

樹形図の図示ということを考えると、今回の図示は枝の数が多すぎます。綺麗な絵を作りたい場合は`dendrogram` に  `truncate_mode` を指定するのが良いでしょう。

In [ ]:
link = linkage(iris.data, method='ward', metric='euclidean')
r = dendrogram(link, labels=iris.target)
plt.show()

戻り値を使って正答率を見てみましょう。今回は `confusion_matrix` を使ってみます。$1 \leftrightarrow 2$ が入れ替わってしまってますが、$0$ については 100% 認識でき、$1, \;2$ に関しては 70% の正解率となっています。

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(iris.target, r['ivl'])

### 課題：

`linkage` の `method` を変えて様子を眺めてください。(`ward` から `single`, `complete` になど。またドキュメントを参照して、`single`, `complete` はどういう定義でクラスタ間の距離を定義しているか確認してください。）

# ChiMerge

クラスタリングとは多少異なりますが、データの離散化(discretization)について考えてみましょう。

今回、iris データの第１成分だけ考えます。データを離散化する場合、一番簡単な方法は等間隔、あるいは等頻度になるように区間を分割する方法です。等間隔に分割する場合は、最小値〜最大値を含む区間を適当に等分します。例えば下記のようです。

In [ ]:
d0 = iris.data[0:50, 0]
d1 = iris.data[50:100, 0]
d2 = iris.data[100:150, 0]
h0 = plt.hist(d0, label = "c=0", bins = 50, range = (4, 9), alpha = 0.5, color = "yellow")
h1 = plt.hist(d1, label = "c=1", bins = 50, range = (4, 9), alpha = 0.5, color = "green")
h2 = plt.hist(d2, label = "c=2", bins = 50, range = (4, 9), alpha = 0.5, color = "red")
plt.show()

等頻度に分割する場合は、`numpy.percentile` が使えます。

In [ ]:
ds = iris.data[:, 0]
np.percentile(ds, [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])

上のグラフの例で、$7.0 \sim 8.0$ の範囲は実のところ一つのビンに纏めてしまっても識別に差し支えないように思われます。
一方で、$5.0 \sim 6.0$ の範囲はデータ識別に必要そうに見えます。
隣接するビンを纏めるか否かの基準として、その２つのビンに差があるか否かを$\chi^2$検定で判断することにしましょう。

詳しくは論文 [ChiMerge: Discretization of Numeric Attributes](http://sci2s.ugr.es/keel/pdf/algorithm/congreso/1992-Kerber-ChimErge-AAAI92.pdf) を参考にしてください。

$i$ をビンを表す添字、$j$ をカテゴリを表す添字、$c_{ij}$ がカウント数、$E_{ij}$ は分割表の期待値とするとき、

$$\chi^2 = \sum_{i=1}^2 \sum_{j=0}^{k-1} \frac{(c_{ij}-E_{ij})^2}{E_{ij}}$$

が閾値以下の場合、二つのビンは区別する必要がないとしてマージします。実際は$\chi^2$が一番小さいセルの組をマージするという、階層型クラスタリングと同じことを行います。

なお、$E_{ij}=0$ となる場合は、$E_{ij}=\epsilon$ とゼロ割りエラーが起きないように小さな値を割り当てます。（こうすることによって、$c_{ij}=0$ の場合はマージされ、$c_{ij} \neq 0$ の場合はマージされません。）

In [ ]:
def chi2_for_two(cs1, cs2):
    """
    args:
        cs1 = [c_0, c_1, ..., c_{k-1}]
        cs2 = [c_0, c_1, ..., c_{k-1}]
    return:
        chi^2
    """
    eps = 1.0E-8 # small number
    cs = [(c1 + c2) for (c1, c2) in zip(cs1, cs2)]
    r1 = sum(cs1)
    r2 = sum(cs2)
    n = r1 + r2
    es1 = [(eps if (c*r1==0) else c*r1/n) for c in cs] # expected for d1
    es2 = [(eps if (c*r2==0) else c*r2/n) for c in cs] # expected for d2
    chi2 = sum([(c-e)*(c-e)/e for (c,e) in zip(cs1, es1)]) + \
        sum([(c-e)*(c-e)/e for (c,e) in zip(cs2, es2)])
    return chi2

def chimerge(rs, cs, max_chi2):
    """
    args:
        k = number of category
        rs = [(min, max)]
        cs = [[c_0, c_1, ..., c_{k-1}]]
        max_chi2 = criteria to merge
    return:
        ([(min,max)], [[c_0, c_1, ..., c_{k-1}]], [chi^2の値])
    """
    if (len(rs)<2): # no need to merge
        print('no need to merge. length='+str(len(data)))
        return data
    else:
        chi2s = [chi2_for_two(cs1, cs2) for (cs1,cs2) in zip(cs[:-1], cs[1:])]
        minchi2 = min(chi2s)
        if (minchi2 > max_chi2):
            print('stop merging. chi2='+str(minchi2)+' > '+str(max_chi2))
            return (rs, cs, chi2s)
        else:
            ichi2s = list(enumerate(chi2s))
            ichi2s.sort(key=lambda t: t[1])
            idx = ichi2s[0][0]
            pre_rs = [r for (i,r) in enumerate(rs) if i<idx]
            post_rs = [r for (i,r) in enumerate(rs) if i>(idx+1)]
            merged_r = (rs[idx][0], rs[idx+1][1])
            pre_rs.append(merged_r)
            pre_rs.extend(post_rs)
            pre_cs = [c for (i,c) in enumerate(cs) if i<idx]
            post_cs = [c for (i,c) in enumerate(cs) if i>(idx+1)]
            merged_c = [(c1+c2) for (c1,c2) in zip(cs[idx],cs[idx+1])] 
            pre_cs.append(merged_c)
            pre_cs.extend(post_cs)
            return chimerge(pre_rs, pre_cs, max_chi2)

上記の `chimerge` は手抜き実装なので再帰呼び出しで実装されていますが、本当はループで実装すべきです。

`chimerge` 用にデータを並べます。

本当は、解析対象データを大きさ順に並べ、各ビンにデータが１件づつになるようにビンを作るのが正しいのですが、今回は手抜きでヒストグラムからデータを作成します。

In [ ]:
rs = [(a,b) for (a,b) in zip(h0[1][:-1], h0[1][1:])] # 範囲
cs = [[c0, c1, c2] for (c0,c1,c2) in zip(h0[0], h1[0], h2[0])] # カウント
#print(rs)
#print(cs)

このデータで実行してみます。$\chi^2(\alpha=0.1, \text{df}=(3-1)(2-1)=2) \sim 4.6$ を使って

In [ ]:
chimerge(rs, cs, 4.6)

ビンは７つ取れば良さそう、と思われます。

### 課題：

`max_chi2` の値を変えるとかして、結果がどう変わるか試してみてください。また、$\chi^2$検定について忘れてしまった人は、復習すると良いでしょう。